## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,73.72,92,100,13.80,overcast clouds
1,1,Busselton,AU,-33.6500,115.3333,58.84,67,83,12.08,broken clouds
2,2,Bahia Blanca,AR,-38.7196,-62.2724,58.37,32,0,10.33,clear sky
3,3,Hermanus,ZA,-34.4187,19.2345,58.80,73,8,4.09,clear sky
4,4,Torbay,CA,47.6666,-52.7314,39.27,94,100,10.00,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,73.72,92,100,13.80,overcast clouds
7,7,Atuona,PF,-9.8000,-139.0333,78.85,71,96,20.71,overcast clouds
8,8,Uige,AO,-7.6087,15.0613,75.72,78,81,2.86,broken clouds
11,11,Guerrero Negro,MX,27.9769,-114.0611,70.45,51,75,5.08,broken clouds
20,20,Benicarlo,ES,40.4165,0.4271,80.91,59,91,7.99,overcast clouds
24,24,Rikitea,PF,-23.1203,-134.9692,76.28,68,0,22.73,clear sky
27,27,Albany,US,42.6001,-73.9662,80.24,64,79,3.00,broken clouds
31,31,Vaini,TO,-21.2000,-175.2000,71.76,73,75,9.22,broken clouds
32,32,Elizabeth City,US,36.2946,-76.2511,73.42,94,75,8.05,broken clouds
33,33,Ponta Do Sol,PT,32.6667,-17.1000,76.71,64,10,10.00,clear sky


In [5]:
len(preferred_cities_df)

250

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,73.72,92,100,13.80,overcast clouds
7,7,Atuona,PF,-9.8000,-139.0333,78.85,71,96,20.71,overcast clouds
8,8,Uige,AO,-7.6087,15.0613,75.72,78,81,2.86,broken clouds
11,11,Guerrero Negro,MX,27.9769,-114.0611,70.45,51,75,5.08,broken clouds
20,20,Benicarlo,ES,40.4165,0.4271,80.91,59,91,7.99,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
691,691,Timmins,CA,48.4669,-81.3331,80.42,41,40,8.05,scattered clouds
695,695,Conakry,GN,9.5380,-13.6773,82.42,80,27,17.27,scattered clouds
697,697,Arroyo,ES,41.6096,-4.7969,71.64,56,0,11.50,clear sky
698,698,Ilheus,BR,-14.7889,-39.0494,82.22,69,75,11.50,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,73.72,overcast clouds,22.0752,-159.3190,
7,Atuona,PF,78.85,overcast clouds,-9.8000,-139.0333,
8,Uige,AO,75.72,broken clouds,-7.6087,15.0613,
11,Guerrero Negro,MX,70.45,broken clouds,27.9769,-114.0611,
20,Benicarlo,ES,80.91,overcast clouds,40.4165,0.4271,
24,Rikitea,PF,76.28,clear sky,-23.1203,-134.9692,
27,Albany,US,80.24,broken clouds,42.6001,-73.9662,
31,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,
32,Elizabeth City,US,73.42,broken clouds,36.2946,-76.2511,
33,Ponta Do Sol,PT,76.71,clear sky,32.6667,-17.1000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,73.72,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Atuona,PF,78.85,overcast clouds,-9.8000,-139.0333,Villa Enata
8,Uige,AO,75.72,broken clouds,-7.6087,15.0613,Hotel Catraio
11,Guerrero Negro,MX,70.45,broken clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
20,Benicarlo,ES,80.91,overcast clouds,40.4165,0.4271,Hotel & Spa Peñíscola Plaza Suites
24,Rikitea,PF,76.28,clear sky,-23.1203,-134.9692,People ThankYou
27,Albany,US,80.24,broken clouds,42.6001,-73.9662,
31,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
32,Elizabeth City,US,73.42,broken clouds,36.2946,-76.2511,Hampton Inn Elizabeth City
33,Ponta Do Sol,PT,76.71,clear sky,32.6667,-17.1000,Hotel do Campo


In [11]:
hotel_df.notnull().sum()

City                   247
Country                247
Max Temp               247
Current Description    247
Lat                    247
Lng                    247
Hotel Name             247
dtype: int64

In [12]:
len(hotel_df)

247

In [13]:
import numpy as np

# 7. Drop the rows where there is no Hotel Name.

c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,73.72,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Atuona,PF,78.85,overcast clouds,-9.8000,-139.0333,Villa Enata
8,Uige,AO,75.72,broken clouds,-7.6087,15.0613,Hotel Catraio
11,Guerrero Negro,MX,70.45,broken clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
20,Benicarlo,ES,80.91,overcast clouds,40.4165,0.4271,Hotel & Spa Peñíscola Plaza Suites


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
#hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))